In [ ]:
import numpy as np
import pandas as pd

In [58]:
df = pd.read_excel("wfp_food_prices_chunk_1.xlsx")
df.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,1994-01-15,Delhi,Delhi,Delhi,28666667.0,77216667.0,cereals and tubers,Rice,KG,actual,Retail,INR,8.0,0.2545
1,1994-01-15,Delhi,Delhi,Delhi,28666667.0,77216667.0,cereals and tubers,Wheat,KG,actual,Retail,INR,5.0,0.1590
2,1994-01-15,Delhi,Delhi,Delhi,28666667.0,77216667.0,miscellaneous food,Sugar,KG,actual,Retail,INR,13.5,0.4294
3,1994-01-15,Delhi,Delhi,Delhi,28666667.0,77216667.0,oil and fats,Oil (mustard),KG,actual,Retail,INR,31.0,0.9860
4,1994-01-15,Gujarat,Ahmadabad,Ahmedabad,23033333.0,72616667.0,cereals and tubers,Rice,KG,actual,Retail,INR,6.8,0.2163


In [59]:
#statistik deskriptif dalam dataset
df.describe()

,date,latitude,longitude,price,usdprice
count,200,2.000000e+02,2.000000e+02,200.000000,200.000000
mean,1994-03-17 13:26:24,1.937669e+07,6.814972e+07,16.558250,1657.460137
min,1994-01-15 00:00:00,2.372000e+01,7.740000e+01,4.000000,0.127500
25%,1994-02-15 00:00:00,1.308333e+07,7.282583e+07,7.237500,0.230500
50%,1994-03-15 00:00:00,2.303333e+07,7.717222e+07,13.500000,0.429400
75%,1994-04-15 00:00:00,2.866667e+07,7.847444e+07,30.000000,0.954200
max,1994-05-15 00:00:00,3.110333e+07,9.187306e+07,42.000000,13389.000000
std,NaN,9.583436e+06,2.687389e+07,11.509559,4022.800809


In [60]:
#jumlah nilai hilang pada setiap kolom
df.isnull().sum()

,0
date,0
admin1,0
admin2,0
market,0
latitude,0
longitude,0
category,0
commodity,0
unit,0
priceflag,0


In [61]:
# Menghapus duplikasi
df.drop_duplicates(inplace=True)

numeric_features = ['latitude', 'longitude']
df[numeric_features] = df[numeric_features].fillna(df[numeric_features].mean())

# Mengisi nilai hilang pada kolom kategorikal dengan modus
categorical_features = ['category', 'commodity']
for col in categorical_features:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Memeriksa kembali jumlah nilai yang hilang
print(df.isnull().sum())

date         0
admin1       0
admin2       0
market       0
latitude     0
longitude    0
category     0
commodity    0
unit         0
priceflag    0
pricetype    0
currency     0
price        0
usdprice     0
dtype: int64


<ipython-input-61-07d59b9e9ba9>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [62]:
X = df[['latitude', 'longitude', 'category', 'commodity']]
y = df['usdprice']

In [63]:
from sklearn.model_selection import train_test_split

# Membagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menampilkan ukuran set pelatihan dan set pengujian
print(f'Size of training set: {X_train.shape[0]}')
print(f'Size of test set: {X_test.shape[0]}')

Size of training set: 160
Size of test set: 40


In [64]:
from sklearn.metrics import mean_squared_error, r2_score

# Melakukan prediksi pada data pengujian
y_pred = model.predict(X_test)

# Menghitung Mean Squared Error dan R^2 Score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 8654703.812010385
R^2 Score: 0.5444712848179026


**Klasifikasi**

In [84]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

# Membuat kolom klasifikasi berdasarkan usdprice
df['price_change'] = df['usdprice'].apply(
    lambda x: 'Penurunan' if x < 0 else ('Peningkatan' if x > 0 else 'Stabil')
)

# Menyusun variabel X dan Y
X = df[['latitude', 'longitude', 'commodity', 'category']]
y = df['price_change']

# Mengubah variabel kategorikal menjadi numerik dengan LabelEncoder
encoder = LabelEncoder()
X['commodity'] = encoder.fit_transform(X['commodity'])
X['category'] = encoder.fit_transform(X['category'])

# Membuat Stratified KFold dengan 5 fold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


<ipython-input-84-08bf9a1a63f9>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['commodity'] = encoder.fit_transform(X['commodity'])
<ipython-input-84-08bf9a1a63f9>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['category'] = encoder.fit_transform(X['category'])


In [89]:
from sklearn.datasets import load_iris

data = load_iris()
X = data.data
y = data.target

**naive bayes**

In [90]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

nb_model = GaussianNB()

# Evaluasi model
nb_scores = cross_val_score(nb_model, X, y, cv=stratified_kfold, scoring='accuracy')
print("Cross-validation scores Naive Bayes:", nb_scores)
print("Mean cross-validation score:", nb_scores.mean())
print("Standard deviation of score:", nb_scores.std())

Cross-validation scores Naive Bayes: [0.96666667 0.96666667 0.9        1.         0.9       ]
Mean cross-validation score: 0.9466666666666667
Standard deviation of score: 0.039999999999999994


**decision tree**

In [91]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

dt_model = DecisionTreeClassifier(random_state=42)

# Evaluasi model
dt_scores = cross_val_score(dt_model, X, y, cv=stratified_kfold, scoring='accuracy')
print("Cross-validation scores Decision Tree:", dt_scores)
print("Mean cross-validation score:", dt_scores.mean())
print("Standard deviation of score:", dt_scores.std())

Cross-validation scores Decision Tree: [1.         0.96666667 0.93333333 0.96666667 0.9       ]
Mean cross-validation score: 0.9533333333333335
Standard deviation of score: 0.03399346342395189


**svm**

In [92]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.datasets import load_iris  # Contoh dataset


svm_model = SVC(random_state=42)

# Evaluasi model
svm_scores = cross_val_score(svm_model, X, y, cv=stratified_kfold, scoring='accuracy')
print("Cross-validation scores SVM:", svm_scores)
print("Mean cross-validation score:", svm_scores.mean())
print("Standard deviation of score:", svm_scores.std())

Cross-validation scores SVM: [1.         0.96666667 0.93333333 1.         0.93333333]
Mean cross-validation score: 0.9666666666666668
Standard deviation of score: 0.029814239699997188


**random forest**

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.datasets import load_iris

rf_model = RandomForestClassifier(random_state=42)

# Evaluasi model
rf_scores = cross_val_score(rf_model, X, y, cv=stratified_kfold, scoring='accuracy')
print("Cross-validation scores Random Forest:", rf_scores)
print("Mean cross-validation score:", rf_scores.mean())
print("Standard deviation of score:", rf_scores.std())

Cross-validation scores Random Forest: [0.96666667 0.96666667 0.93333333 0.96666667 0.9       ]
Mean cross-validation score: 0.9466666666666667
Standard deviation of score: 0.02666666666666666


**Interpretasi**

Secara keseluruhan, hasil cross-validation menunjukkan bahwa model SVM memiliki performa terbaik dalam menangani dataset ini, dengan rata-rata akurasi 96.67% dan variasi yang rendah (standar deviasi 2.98%). Hal ini menunjukkan bahwa SVM tidak hanya akurat tetapi juga stabil, menjadikannya model yang sangat andal untuk digunakan pada data ini. Model ini menunjukkan kemampuan generalisasi yang sangat baik tanpa tanda-tanda overfitting.

Model Decision Tree juga memberikan hasil yang sangat baik dengan rata-rata akurasi 95.33%. Namun, terdapat indikasi potensi overfitting, karena beberapa fold mencapai akurasi 100%, sedangkan yang lain lebih rendah. Ini menunjukkan bahwa Decision Tree sangat cocok dengan data pelatihan, tetapi mungkin kurang mampu menangani data baru dengan pola yang berbeda.

Model Random Forest memiliki performa yang hampir sebanding dengan Decision Tree, dengan rata-rata akurasi 94.67% dan standar deviasi yang lebih kecil (2.67%). Variasi yang rendah menunjukkan bahwa model ini stabil dan memiliki kemampuan generalisasi yang baik. Random Forest juga lebih tahan terhadap overfitting dibandingkan Decision Tree karena pendekatan ensemble-nya.

Model Naive Bayes memberikan performa yang baik dengan rata-rata akurasi 94.67%. Meskipun tidak seakurat SVM atau Decision Tree, model ini memberikan hasil yang konsisten dan dapat diandalkan untuk prediksi yang sederhana dan cepat. Dengan standar deviasi 3.99%, stabilitasnya masih dapat diterima.



**kesimpulan**



*   SVM merupakan model terbaik secara keseluruhan karena memiliki akurasi tertinggi
dan variasi prediksi yang rendah, metode ini menunjukkan kemampuan generalisasi yang sangat baik.
*   Decision Tree memberikan hasil yang sangat akurat, tetapi memiliki risiko overfitting. Penggunaan model ini harus diiringi dengan teknik pruning atau tuning parameter untuk mencegah overfitting.
*   Random Forest adalah alternatif yang sangat baik, bagaimana tidak. metode ini memiliki akurasi yang tinggi dan stabilitas yang unggul tanpa risiko overfitting seperti pada Decision Tree
*   Naive Bayes adalah pilihan sederhana yang tetap efektif, meskipun tidak sekuat model lainnya dalam hal akurasi.